In [1]:
%matplotlib qt
%matplotlib qt
from scipy import interpolate
import scipy.constants as const
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.offsetbox import AnchoredText
from matplotlib.widgets import Slider
from matplotlib.widgets import Button as mat_Button

from Helpful_functions import *
from Curvature_Kspace_functions import *

# cwd = os.getcwd()
cwd = 'C:\\Users\\amorf\\Documents\\Oxford\\DiamondLS\\Wans_Python_Package'

In [13]:
def Redo_data1_plot():
    global X, Y, Z, ax_data1, state, xlims, ylims
    ax_data1.clear()
    ax_data1.contourf(X, Y, Z, 150, cmap='Greys')
    ax_data1.axhline(0, color='1', ls='--', lw=0.8)
    ax_data1.set_ylim(ylims)
    ax_data1.set_xlim(xlims)
    ax_data1.set_title(Zstate+' - '+state)
    plt.pause(0.01)
    
def Redo_data2_plot():
    global X, Y, Z, state, ax_data1, ax_data2, ax_Slice, X_Points, Y_Points
    global k, E, dk, dE, colors, index, data_sets
    ax_data2.clear()
    if state == 'EDC':
        ax_data2.fill_between([k-dk/2,k+dk/2], [max(Y),max(Y)], [min(Y),min(Y)], facecolor='tab:red', alpha=0.2)
    if state == 'MDC':
        ax_data2.fill_between([min(X), max(X)], [E+dE/2, E+dE/2], [E-dE/2, E-dE/2], facecolor='tab:red', alpha=0.2)
    for j in data_sets.keys():
        if j == index:
            ax_data2.scatter(X_Points, Y_Points, marker='*', color=colors[index])
        else:
            x, y = data_sets[j]
            ax_data2.scatter(x, y, marker='*', color=colors[j])
    ax_data2.set_ylim(ax_data1.get_ylim())
    ax_data2.set_yticks([])
    plt.pause(0.01)
    
def Redo_Slice_plot():
    global X, Y, Z, state, ax_data1, ax_data2, ax_Slice, X_Points, Y_Points
    global k, E, dk, dE, Slice_smooth, smooth_win_X, smooth_win_Y, step_X, step_Y
    ax_Slice.clear()
    if state == 'EDC':
        mask = (k-dk/2<X)&(X<k+dk/2)
        Slice = np.mean(Z[:,mask], axis=1)
        Slice_smooth = savgol_filter(Slice, smooth_win_Y, 3)
        ax_Slice.plot(Slice, Y, color='tab:blue')
        ax_Slice.plot(Slice_smooth, Y+step_Y/2, color='0', ls='--')
        ax_Slice.set_ylim(ax_data1.get_ylim())
    if state == 'MDC':
        mask = (E-dE/2<Y)&(Y<E+dE/2)
        Slice = np.mean(Z[mask,:], axis=0)
        Slice_smooth = savgol_filter(Slice, smooth_win_X, 3)
        ax_Slice.plot(X, Slice, color='tab:blue')
        ax_Slice.plot(X+step_X/2, Slice_smooth, color='0', ls='--')
    ax_Slice.set_yticks([])
    ax_Slice.set_title(f'k = {np.round(k, 3)}, E = {np.round(E)}')
    plt.pause(0.01)

def change_to_raw(var):
    global k_red, E_red, k_data_red, X, Y, Z, Zstate
    Zstate = 'Raw'
    X, Y, Z = k_red, E_red, k_data_red
    Redo_data1_plot()
    Redo_Slice_plot()
    
def change_to_cur(var):
    global X_C1D_E, Y_C1D_E, Z_C1D_E, X_C1D_M, Y_C1D_M, Z_C1D_M
    global X, Y, Z, state, Zstate

    Zstate = 'Curv'
    if state == 'MDC':
        X, Y, Z = X_C1D_M, Y_C1D_M, Z_C1D_M
    else:
        X, Y, Z = X_C1D_E, Y_C1D_E, Z_C1D_E
    Redo_data1_plot()
    Redo_Slice_plot()
    
def change_to_EDC(var):
    global X_C1D_E, Y_C1D_E, Z_C1D_E
    global X, Y, Z, Zstate, state
    state = 'EDC'
    if Zstate == 'Curv':
        X, Y, Z = X_C1D_E, Y_C1D_E, Z_C1D_E
    Redo_data1_plot()
    Redo_data2_plot()
    Redo_Slice_plot()
        
def change_to_MDC(var):
    global X_C1D_M, Y_C1D_M, Z_C1D_M
    global X, Y, Z, Zstate, state
    state = 'MDC'
    if Zstate == 'Curv':
        X, Y, Z = X_C1D_M, Y_C1D_M, Z_C1D_M
    Redo_data1_plot()
    Redo_data2_plot()
    Redo_Slice_plot()
    
def find_peak(x):
    global Zstate
    if Zstate == 'Raw':
        return np.max(x)
    else:
        return np.min(x)

def curv_finder(event):
    global X, Y, Z, state, ax_data1, ax_data2, ax_Slice, X_Points, Y_Points
    global Slice_smooth, k, E, dk, dE, step_X, step_Y
    if event.inaxes.get_label() == 'Left':
        k, E = round(event.xdata/dk)*dk, round(event.ydata/dE)*dE
        Redo_data2_plot()
        Redo_Slice_plot()
        
    if event.inaxes.get_label() == 'Right':
        if state == 'EDC':
            E_guess = event.ydata
            mask = (E_guess-dE<Y)&(Y<E_guess+dE)
            Slice_Point = find_peak(Slice_smooth[mask])
            E_Point = Y[Slice_smooth == Slice_Point][0] + step_Y/2
            ax_Slice.scatter(Slice_Point, E_Point, marker='*', color='0')
            X_Points.append(k)
            Y_Points.append(E_Point)
        else:
            k_guess = event.xdata
            mask = (k_guess-dk<X)&(X<k_guess+dk)
            Slice_Point = find_peak(Slice_smooth[mask])
            k_Point = X[Slice_smooth == Slice_Point][0] + step_X/2 
            ax_Slice.scatter(k_Point, Slice_Point, marker='*', color='0')
            X_Points.append(k_Point)
            Y_Points.append(E)
        Redo_data2_plot()
        
def undo_move(event):
    global k, E, dk, dE, X_Points, Y_Points
    if event.key == 'backspace':
        X_Points = X_Points[:-1]
        Y_Points = Y_Points[:-1]
        Redo_data2_plot()
    if event.key == 'left':
        if state == 'EDC':
            k = round(k - dk, 3)
            Redo_data2_plot()
            Redo_Slice_plot()
    if event.key == 'right':
        if state == 'EDC':
            k = round(k + dk, 3)
            Redo_data2_plot()
            Redo_Slice_plot()
    if event.key == 'up':
        if state == 'MDC':
            E = round(E + dE)
            Redo_data2_plot()
            Redo_Slice_plot()
    if event.key == 'down':
        if state == 'MDC':
            E = round(E - dE)
            Redo_data2_plot()
            Redo_Slice_plot()
    
def band_change(event, band_no):
    global index, colors, data_sets, X_Points, Y_Points
    global indicator_ax
    if band_no == 4:
        data_sets[index] = [X_Points, Y_Points]
        plt.close()
    else:
        indicator_ax.set_facecolor(colors[band_no])
        # save data to previos index
        data_sets[index] = [X_Points, Y_Points]
        # collect data from new index
        index = band_no
        X_Points, Y_Points = data_sets[band_no]
        Redo_data2_plot()
        plt.pause(0.01)

### Load the Data

In [14]:
file_name = 'i05-157361.nxs'
Ef = 53.544154223013706
offset = -0.832994129032258

# use os.chdir to go to where the file is stored
# os.chdir(directory)

with h5py.File(file_name, 'r') as I05:
    T = sigfig(np.squeeze(I05['entry1/sample/cryostat_temperature'][:]), 3)
    data_2D = np.squeeze(I05['entry1/analyser/data'][:]).T
    angles = np.squeeze(I05['entry1/analyser/angles'][:])
    energies = np.squeeze(I05['entry1/analyser/energies'][:])
    
data_2D = dead_pixels(data_2D, 20)

In [15]:
k_full, E_full, k_data_full = convert2kspace(angles, energies, data_2D.T, Ef, offset)

# dE, dk, xlims and ylims can all be editted
dk, dE = 0.02, 4 
xlims, ylims = (-0.6, 0.6), (-150, 50)

k_red, E_red, k_data_red = crop_data(k_full, E_full, k_data_full, xlims, ylims)

XYZ_M, XYZ_E, XYZ_ME = reduce_k_data(k_full, E_full, k_data_full, dk=dk, dE=dE)

X_C1D_M, Y_C1D_M, Z_C1D_M, win_k = curvature_1D_M(XYZ_M[0], XYZ_M[1], XYZ_M[2], f=5, dk=dk)
X_C1D_M, Y_C1D_M, Z_C1D_M = crop_data(X_C1D_M, Y_C1D_M, Z_C1D_M, xlims, ylims)
X_C1D_E, Y_C1D_E, Z_C1D_E, win_k = curvature_1D_E(XYZ_E[0], XYZ_E[1], XYZ_E[2], f=5, dE=dE)
X_C1D_E, Y_C1D_E, Z_C1D_E = crop_data(X_C1D_E, Y_C1D_E, Z_C1D_E, xlims, ylims)

X_C2D, Y_C2D, Z_C2D, win = curvature_2D(XYZ_ME[0], XYZ_ME[1], XYZ_ME[2], win=6)
X_C2D, Y_C2D, Z_C2D = crop_data(X_C2D, Y_C2D, Z_C2D, xlims, ylims)

In [16]:
# Check it
XYZs = [
    [k_red, E_red, k_data_red],
    [X_C1D_M, Y_C1D_M, Z_C1D_M],
    [X_C1D_E, Y_C1D_E, Z_C1D_E],
    [X_C2D, Y_C2D, Z_C2D],
]

for XYZ in XYZs:
    fig, ax = plt.subplots()
    ax.contourf(XYZ[0], XYZ[1], XYZ[2], 100, cmap='Greys')
    plt.show()

### Collect data

In [17]:
print('Would you like to use existing data_sets? (y/n)')
answer = str(input())
if answer == 'y':
    pass
else:
    data_sets = {0:[[],[]], 1:[[],[]], 2:[[],[]], 3:[[],[]]}
    
index = 0
# dk = 0.015
# dE = 2
X, Y, Z = k_red, E_red, k_data_red
k, E = 0, 0

step_X = np.mean(X[1:]-X[:-1])
step_Y = np.mean(Y[1:]-Y[:-1])
smooth_win_X = 4+int(len(X)/100)
smooth_win_Y = 4+int(len(Y)/100)

Zstate = 'Raw'
state = 'EDC'

X_Points, Y_Points = data_sets[index]
colors = ['tab:red', 'tab:green', 'tab:blue', 'tab:olive']

fig, axs = plt.subplots(1,2, figsize=(12,6.5))
ax_data1, ax_Slice = axs
ax_data2 = ax_data1.twinx()
ax_data1.set_label('Left')
ax_data2.set_label('Left')
ax_Slice.set_label('Right')
ax_data1.contourf(X, Y, Z, 150, cmap='Greys')
ax_data1.axhline(0, color='1', ls='--', lw=0.8)
ax_data1.set_ylim(ylims)
ax_data1.set_xlim(xlims)
ax_data1.set_title(Zstate+' - '+state)
ax_Slice.set_yticks([])
ax_data2.set_ylim(ax_data1.get_ylim())
ax_Slice.set_ylim(ax_data1.get_ylim())
ax_data2.set_yticks([])
Redo_data2_plot()
Redo_Slice_plot()
plt.subplots_adjust(bottom=0.2)

fig.canvas.mpl_connect('button_press_event', curv_finder)
fig.canvas.mpl_connect('key_press_event', undo_move)

#Add Button to switch from raw to curvature
raw_ax = plt.axes([0.15,0.02,0.12,0.08])
cur_ax = plt.axes([0.3,0.02,0.12,0.08])
raw_Btn = mat_Button(raw_ax, 'Raw', color='0.8', hovercolor='0.7')
cur_Btn = mat_Button(cur_ax, 'Curv', color='0.8', hovercolor='0.7')
raw_Btn.on_clicked(change_to_raw)
cur_Btn.on_clicked(change_to_cur)

#Add Buttons for EDC or MDC
EDC_ax = plt.axes([0.6,0.02,0.12,0.08])
MDC_ax = plt.axes([0.75,0.02,0.12,0.08])
EDC_Btn = mat_Button(EDC_ax, 'EDC', color='0.8', hovercolor='0.7')
MDC_Btn = mat_Button(MDC_ax, 'MDC', color='0.8', hovercolor='0.7')
EDC_Btn.on_clicked(change_to_EDC)
MDC_Btn.on_clicked(change_to_MDC)

#Add Band Indicator
indicator_ax = plt.axes([0.13,0.935,0.04,0.04])
indicator_ax.set_facecolor('tab:red')
indicator_ax.set_xticks([])
indicator_ax.set_yticks([])
#Add Band Button
band0_ax = plt.axes([0.335,0.93,0.05,0.05])
band0_Btn = mat_Button(band0_ax, 'B 0', color=colors[0], hovercolor='0.9')
band0_Btn.on_clicked(lambda event: band_change(event, 0))
band1_ax = plt.axes([0.435,0.93,0.05,0.05])
band1_Btn = mat_Button(band1_ax, 'B 1', color=colors[1], hovercolor='0.9')
band1_Btn.on_clicked(lambda event: band_change(event, 1))
band2_ax = plt.axes([0.535,0.93,0.05,0.05])
band2_Btn = mat_Button(band2_ax, 'B 2', color=colors[2], hovercolor='0.9')
band2_Btn.on_clicked(lambda event: band_change(event, 2))
band3_ax = plt.axes([0.635,0.93,0.05,0.05])
band3_Btn = mat_Button(band3_ax, 'B 3', color=colors[3], hovercolor='0.9')
band3_Btn.on_clicked(lambda event: band_change(event, 3))

#Add Band Button
finish_ax = plt.axes([0.85,0.93,0.05,0.05])
finish_Btn = mat_Button(finish_ax, 'Finish', color='0.8', hovercolor='0.7')
finish_Btn.on_clicked(lambda event: band_change(event, 4))

plt.show()

Would you like to use existing data_sets? (y/n)
n


In [19]:
print(data_sets)

{0: [[0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.14, 0.16], [-2.189223013704745, -2.189223013704745, -1.8092230137049201, -2.189223013704745, -2.56922301370457, -69.06922301370236, -68.68922301370253, -68.3092230137027, -70.20922301370183, -71.72922301370824]], 1: [[], []], 2: [[], []], 3: [[], []]}
